In [1]:
import polars as pl
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [12]:
def startTrain(X, y, criterion, bootstrap, max_depth, min_samples_split, n_estimators):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        rus = RandomUnderSampler(random_state=42)
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)

        rf = RandomForestClassifier(
            criterion=criterion, bootstrap=bootstrap, max_depth=max_depth,
            min_samples_split=min_samples_split, n_estimators=n_estimators
        )

        rf.fit(X_train_resampled, y_train_resampled)
        y_pred = rf.predict(X_test_scaled)

        # Avaliação
        accuracy = accuracy_score(y_test, y_pred)
        results_fold.append(accuracy)

    return np.mean(results_fold)

In [ ]:
def grid_search():
    param_grid = {
        'criterion': ['gini', 'entropy'],  
        'bootstrap': [True, False],       
        'max_depth': [10, 20, 30, None],  
        'min_samples_split': [2, 3, 5],   
        'n_estimators': [50, 100, 200]    
    }
    
    best_accuracy = 0
    best_params = {}

    for criterion in param_grid['criterion']:
        for bootstrap in param_grid['bootstrap']:
            for max_depth in param_grid['max_depth']:
                for min_samples_split in param_grid['min_samples_split']:
                    for n_estimators in param_grid['n_estimators']:
                        accuracy = startTrain(
                            X, y, criterion=criterion, bootstrap=bootstrap,
                            max_depth=max_depth, min_samples_split=min_samples_split,
                            n_estimators=n_estimators
                        )

                        print(f"criterion={criterion}, bootstrap={bootstrap}, max_depth={max_depth}, min_samples_split={min_samples_split}, n_estimators={n_estimators} -> Accuracy: {accuracy}")

                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params = {
                                'criterion': criterion,
                                'bootstrap': bootstrap,
                                'max_depth': max_depth,
                                'min_samples_split': min_samples_split,
                                'n_estimators': n_estimators
                            }
    
    print("\nMelhores Hiperparâmetros encontrados:", best_params)
    print(f"Melhor Acurácia Média: {best_accuracy}")
    return best_params


In [14]:
inicio = time.time()
best_params = grid_search()
fim = time.time()

criterion=gini, bootstrap=True, max_depth=10, min_samples_split=2, n_estimators=50 -> Accuracy: 0.9949804852287294
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=2, n_estimators=100 -> Accuracy: 0.9949606737326582
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=2, n_estimators=200 -> Accuracy: 0.994989490359852
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=3, n_estimators=50 -> Accuracy: 0.9949714800002936
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=3, n_estimators=100 -> Accuracy: 0.9949822862679291
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=3, n_estimators=200 -> Accuracy: 0.9949912914314893
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=5, n_estimators=50 -> Accuracy: 0.994980485163854
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=5, n_estimators=100 -> Accuracy: 0.994960673748877
criterion=gini, bootstrap=True, max_depth=10, min_samples_split=5, n_estimator

In [15]:
duracao = fim - inicio
print(duracao)

12133.26722073555


In [16]:
print("\nMelhores Hiperparâmetros encontrados:", best_params)


Melhores Hiperparâmetros encontrados: {'criterion': 'entropy', 'bootstrap': False, 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}


In [17]:
for i in range(1,31):
    startTrain()
    print(i)

TypeError: startTrain() missing 7 required positional arguments: 'X', 'y', 'criterion', 'bootstrap', 'max_depth', 'min_samples_split', and 'n_estimators'

In [ ]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

In [ ]:
metrics_df.write_csv("metrics_results/unbalanced_rf_metrics_output.csv", separator=';')